In [17]:
import numpy as np
import torch
import torch.nn as nn      
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import cv2
from PIL import Image
from pylab import rcParams
rcParams['figure.figsize'] = 5, 5
import tqdm
from numpy import moveaxis
import scipy.ndimage
from IPython.display import clear_output
import time
from mss.linux import MSS as mss
import mss.tools

animals= ["chicken", "pig", "panda", "terrain"]

In [18]:
def screen_cap(top = 0, left = 0, width = 650, height = 480):
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    img = np.array(sct.grab(mon))[...,:3]
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sct.close()
    return img

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 7, 5)
        self.pool = nn.MaxPool2d(3, 3)
        
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(7, 10, 5)
        self.fc1 = nn.Linear(90, 60)
        self.fc2 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(30, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
hunter = Net()
hunter.load_state_dict(torch.load("hunter.pt"))
hunter.eval()

Net(
  (conv1): Conv2d(3, 7, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(7, 10, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=90, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=4, bias=True)
)

In [20]:
def generate_heatmap_numpy(input_np_arr, model, kernel_size, stride, thresh_prob, label):
    image = input_np_arr
    width, height = image.shape[1], image.shape[0]
    #setting the output image width and height
    output_height = int(np.ceil((height-kernel_size)/stride))
    output_width = int(np.ceil((width-kernel_size)/stride))
    #create a white image of sizes we defined
    heatmap = np.zeros((output_height, output_width))
    for h in range(0, height):
        for w in range(0, width):
            h_start = h*stride
            w_start = w*stride
            h_end = min(height, h_start + kernel_size)
            w_end = min(width, w_start + kernel_size)
            if (w_end) >= width or (h_end) >= height:
                continue
            np_slice = image[h_start:h_end, w_start:w_end, :]
            if 0 in np_slice.shape:
                continue
            np_slice = cv2.resize(np_slice, (50,50))
            image_moveaxis =  np.moveaxis(np_slice, 2, 0)    
            sweet_tensor =  torch.from_numpy(image_moveaxis).unsqueeze(0).float()
            output = model(sweet_tensor).flatten()
            output = torch.softmax(output, dim = 0)[label].item()
            if  output > thresh_prob:
                heatmap[h,w] = output  # mult
            else:
                heatmap[h,w] =  0 
    return heatmap

In [23]:
def look_for_animals_and_save_image(input_np_arr, model, kernel_size, stride, thresh_prob, label, folder,  filename  ):
    image = input_np_arr
    width, height = image.shape[1], image.shape[0]

    for h in range(0, height):
        for w in range(0, width):
            h_start = h*stride
            w_start = w*stride
            h_end = min(height, h_start + kernel_size)
            w_end = min(width, w_start + kernel_size)
            if (w_end) >= width or (h_end) >= height:
                continue
            np_slice = image[h_start:h_end, w_start:w_end, :]
            if 0 in np_slice.shape:
                continue
            np_slice = cv2.resize(np_slice, (50,50))
            image_moveaxis =  np.moveaxis(np_slice, 2, 0)    
            sweet_tensor =  torch.from_numpy(image_moveaxis).unsqueeze(0).float()
            output = model(sweet_tensor).flatten()
            output = torch.softmax(output, dim = 0)[label].item()
            if  output > thresh_prob:
                
                filename = folder + str(int(round(time.time(), 2)*100)) + "_" + filename
                time.sleep(0.2)
                cv2.imwrite(filename, np_slice)
                print("saved --  ", filename)
            
            else:
                pass 
   

In [22]:
 def detect_animal_numpy(model, image_np_array, kernel_size, stride, label, threshold_prob, purpose = "heatmap", folder = "raw_images/chicken/", filename_if_save = "chicken.png"):
    
    im = image_np_array
    
    if purpose == "heatmap":
        
        heatmap = generate_heatmap_numpy(im, model, kernel_size, stride, threshold_prob, label)

        heat_shape = im.shape[:-1][::-1]
        heatmap = cv2.resize(heatmap,heat_shape).astype(np.float32)
        heatmap = np.stack((heatmap*0, heatmap*0, heatmap*150), axis=-1)  ## force convert image to rgb    
        fin = cv2.add(im.astype(np.float32),heatmap)
        m = fin.astype(np.uint8)


        return m
    else:
        
         look_for_animals_and_save_image(im, model, kernel_size, stride, threshold_prob, label, filename = filename_if_save, folder = folder)
         

In [1]:
while True:
    
    try:
        start = time.time()
        frame = screen_cap(top= 140, height = 410)
        frame2 = cv2.resize(frame, (int(frame.shape[1]/4), int(frame.shape[0]/4)))
        frame2 = detect_animal_numpy(hunter,frame2, 40, 15, 2, 0.95,  purpose = "collect_images", folder = "raw_images/dummy/", filename_if_save = "panda.png")
    
        time_taken = time.time() - start

        print(int(1/time_taken), " FPS")
        clear_output(wait = True)
    except:
        print("end")
        break


end
